# 5.0 Introduction

衡量物体的质量而不是数量通常是有用的。我们经常将这些定性信息表示为一个离散类别中的观察成员，例如性别、颜色或汽车品牌。然而，并非所有的分类数据都是相同的。没有内在顺序的类别集称为名词性的。名义类别的例子包括：              
•蓝色、红色、绿色\•男士、女士\•香蕉、草莓、苹果              
相反，当一组类别具有某种自然顺序时，我们称之为序数。例如：              
•低、中、高\•年轻人、老年人\•同意、中立、不同意             
此外，分类信息通常在数据中表示为字符串的向量或列（例如，“缅因州”、“德克萨斯州”、“特拉华州”）。
问题是大多数机器学习算法要求输入是数值。              
k-最近邻算法提供了一个简单的例子。算法的一个步骤是使用欧几里德距离计算观测值之间的距离

$$
 \sqrt{\Sigma_{i=1}^n(x_i − y_i)^2}
$$

where $x$ and $y$ are two observations and subscript $i$ denotes the value for the observations’ $i$th feature. However, the distance calculation obviously is impossible if the value of $x_i$ is a string (e.g., `"Texas"`). Instead, we need to convert the string into some numerical format so that it can be inputted into the Euclidean distance equation. Our goal is to make a transformation that properly conveys the information in the categories (ordinality, relative intervals between categories, etc.). In this chapter we will cover techniques for making this transformation as well as overcoming other challenges often encountered when handling categorical data.


# 5.1 Encoding Nominal Categorical Feature

**Problem:**\
You have a feature with nominal classes that has no intrinsic ordering (e.g., apple, pear, banana).

**Solution:**\
One-hot encode the feature using scikit-learn’s `LabelBinarizer`:

In [5]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer

feature = np.array([
    ["Texas"],
    ["California"],
    ["Texas"],
    ["Delaware"],
    ["Texas"]
])

# create one-hot encoder
one_hot = LabelBinarizer()

# one-hot encode feature
one_hot.fit_transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [3]:
# view feature classes
one_hot.classes_
one_hot.transform(feature)

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1]])

In [6]:
# reverse one-hot encoding
one_hot.inverse_transform(one_hot.transform(feature))

array(['Texas', 'California', 'Texas', 'Delaware', 'Texas'], dtype='<U10')

In [7]:
import pandas as pd

pd.get_dummies(feature[:, 0])

,California,Delaware,Texas
0,0,0,1
1,1,0,0
2,0,0,1
3,0,1,0
4,0,0,1


In [8]:
# create multiclass feature
multiclass_feature = [
    ("Texas", "Florida"),
    ("California", "Alabama"),
    ("Texas", "Florida"),
    ("Delaware", "Florida"),
    ("Texas", "Alabama")
]

# create multiclass one-hot encoder
one_hot_multiclass = MultiLabelBinarizer()

# one-hot encode multiclass feature
one_hot_multiclass.fit_transform(multiclass_feature)

array([[0, 0, 0, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 1, 0],
       [1, 0, 0, 0, 1]])

In [9]:
# view classes
one_hot_multiclass.classes_

array(['Alabama', 'California', 'Delaware', 'Florida', 'Texas'],
      dtype=object)

### Discussion
我们可能认为正确的策略是给每个类分配一个数值（例如德克萨斯州=1，加利福尼亚州=2）。然而，当我们的类没有内在的顺序时（例如德克萨斯州并不比加利福尼亚州“少”），我们的数值错误地创建了一个不存在的顺序。              正确的策略是为原始特性中的每个类创建一个二进制特性。这通常被称为一个热编码（在机器学习文献中）或dummying（在统计和研究文献中）。我们的解决方案的特点是包含三个类的向量（即德克萨斯州、加利福尼亚州和特拉华州）。在一个热编码中，每个类都成为自己的特征，当类出现时为1，否则为0。因为一个特性有三个类，所以一个热编码返回三个二进制特性（每个类一个）。通过使用一种热编码，我们可以捕获类中某个观测的成员，同时保持类缺少任何类型的层次结构的概念。              最后，值得注意的是，通常建议在对一个特征进行一次热编码后，我们将其中一个热编码的特征丢弃到结果矩阵中，以避免线性相关性

### See Also
* Dummy Variable Trap, Algosome (http://www.algosome.com/articles/dummy-variable-trap-regression.html)
* Dropping one of the columns when using one-hot encoding, CrossValidated (https://stats.stackexchange.com/questions/231285/dropping-one-of-the-columns-when-using-one-hot-encoding)

## 5.2 Encoding Ordinal Categorical Features

**Problem:**\
You have an ordinal categorical feature (e.g., high, medium, low).


**Solution:**\
Use pandas DataFrame’s replace method to transform string labels to numerical equivalents:


In [14]:
import pandas as pd

# create features
df = pd.DataFrame({"Score": ["Low", "Low", "Medium", "Medium", "High"]})
df

# create mapper
scale_mapper = {
    "Low": 1,
    "Medium": 2,
    "High": 3
}

# replace feature values with scale
df["Score"].replace(scale_mapper)

0    1
1    1
2    2
3    2
4    3
Name: Score, dtype: int64

## 5.3 Encoding Dictionaries of Features

**Problem:**\
You have a dictionary and want to convert it into a feature matrix.

**Solution:**\
Use `DictVectorizer`:

In [15]:
from sklearn.feature_extraction import DictVectorizer

data_dict = [
    {"Red": 2, "Blue": 4},
    {"Red": 4, "Blue": 3},
    {"Red": 1, "Yellow": 2},
    {"Red": 2, "Yellow": 2}
]

# create dictionary vectorizer
dictvectorizer = DictVectorizer(sparse=False)

# convert dictionary to feature matrix
features = dictvectorizer.fit_transform(data_dict)

features

array([[4., 2., 0.],
       [3., 4., 0.],
       [0., 1., 2.],
       [0., 2., 2.]])

In [16]:
# get feature names
dictvectorizer.get_feature_names()

['Blue', 'Red', 'Yellow']

## 5.4 Imputing Missing Class Values

**Problem:**\
You have a categorical feature containing missing values that you want to replace with predicted values.

**Solution:**\
The ideal solution is to train a machine learning classifier algorithm to predict the missing values, commonly a k-nearest neighbors (KNN) classifier:


In [18]:
# load libraries
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

X = np.array([[0, 2.10, 1.45],
            [1, 1.18, 1.33],
            [0, 1.22, 1.27],
            [1, -0.21, -1.19]])

X_with_nan = np.array([[np.nan, 0.87, 1.31],
                      [np.nan, -0.67, -0.22]])

# train KNN learner
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X[:,1:], X[:, 0])

# predict missing values' class
imputed_values = trained_model.predict(X_with_nan[:, 1:])
imputed_values
# join column of predicted class with their other features
X_with_imputed = np.hstack((imputed_values.reshape(-1, 1), X_with_nan[:, 1:]))

# join two feature matricies
np.vstack((X_with_imputed, X))

array([[ 0.  ,  0.87,  1.31],
       [ 1.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

### reshape(-1, 1)代表生成一列数据，对行不做指定

In [20]:
from sklearn.preprocessing import Imputer

# join the two feature matricies
X_complete = np.vstack((X_with_nan, X))
X_complete

imputer = Imputer(strategy='most_frequent', axis=0)

imputer.fit_transform(X_complete)

array([[ 0.  ,  0.87,  1.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

### See Also
* Overcoming Missing Values in a Random Forest Classifier (https://medium.com/airbnb-engineering/overcoming-missing-values-in-a-random-forest-classifier-7b1fc1fc03ba)
* A Study of K-Nearest Neighbour as an Imputation Method (http://conteudo.icmc.usp.br/pessoas/gbatista/files/his2002.pdf)

## 5.5 Handling Imbalanced Classes

**Problem:**\
You have a target vector with highly imbalanced classes.

**Solution:**\

Collect more data. If that isn’t possible, change the metrics used to evaluate your model. If that doesn’t work, consider using a model’s built-in class weight parameters (if available), downsampling, or upsampling. We cover evaluation metrics in a later chapter, so for now let us focus on class weight parameters, downsampling, and upsampling.

To demonstrate our solutions, we need to create some data with imbalanced classes. Fisher’s Iris dataset contains three balanced classes of 50 observations, each indicating the species of flower (*Iris setosa*, *Iris virginica*, and *Iris versicolor*). 

To unbalance the dataset, we remove 40 of the 50 *Iris setosa* observations and then merge the *Iris virginica* and *Iris versicolor* classes. The end result is a binary target vector indicating if an observation is an *Iris setosa* flower or not. The result is 10 observations of *Iris setosa* (class 0) and 100 observations of not *Iris setosa* (class 1):


In [22]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

iris = load_iris()

features = iris.data
target = iris.target

# remove first 40 observations
features = features[40:, :]
target = target[40:]

# create binary target vector indicating if class 0
target = np.where((target == 0), 0, 1)

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
# create weights
weights = {0: .9, 1: 0.1}

RandomForestClassifier(class_weight=weights)

RandomForestClassifier(bootstrap=True, class_weight={0: 0.9, 1: 0.1},
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [24]:
RandomForestClassifier(class_weight="balanced")

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [30]:
i_class0 = np.where(target == 0)[0]
i_class1 = np.where(target == 1)[0]
# 输出满足条件的元素的坐标，[0]代表只输出行

n_class0 = len(i_class0)
n_class1 = len(i_class1)
 
i_class1_downsampled = np.random.choice(i_class1, size=n_class0, replace=False)
#从数组中随机抽取元素，第一个参数表示从中抽取元素，第二个表示抽取个数，第三个false表示不能抽取相同的
np.hstack((target[i_class0], target[i_class1_downsampled]))


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [32]:
np.vstack((features[i_class0,:], features[i_class1_downsampled, :]))[0:5]

array([[5. , 3.5, 1.3, 0.3],
       [4.5, 2.3, 1.3, 0.3],
       [4.4, 3.2, 1.3, 0.2],
       [5. , 3.5, 1.6, 0.6],
       [5.1, 3.8, 1.9, 0.4]])

In [33]:
i_class0_upsampled = np.random.choice(i_class0, size=n_class1, replace=True)

np.concatenate((target[i_class0_upsampled], target[i_class1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

In [34]:
np.vstack((features[i_class0_upsampled,:], features[i_class1,:]))[0:5]

array([[5.3, 3.7, 1.5, 0.2],
       [4.6, 3.2, 1.4, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [5.3, 3.7, 1.5, 0.2],
       [5.3, 3.7, 1.5, 0.2]])

## Discussion

在现实世界中，不平衡的课程随处可见，大多数访问者不点击购买按钮，许多类型的癌症是罕见的。因此，处理不平衡的类是机器学习中常见的活动。
我们最好的策略就是收集更多的观察，尤其是类别较少的观察。然而，这往往是不可能的，所以我们不得不求助于其他选择。

第二种策略是使用更适合不平衡类的模型评估度量。精度通常被用作评估模型性能的度量，但是当存在不平衡类时，精度可能不适合。例如，如果只有0.5%的观察结果患有某种罕见的癌症，那么即使是一个预测没有人患癌症的天真模型也会有99.5%的准确率。显然这并不理想。我们在后面几章讨论的一些更好的指标是混淆矩阵、精确度、召回率、F1分数和ROC曲线。

第三种策略是使用一些模型实现中包含的类权重参数。这允许我们调整算法以适应不平衡的类。幸运的是，许多scikit-learn分类器都有一个“class_-weight”参数，这使它成为一个不错的选择。

第四和第五种策略是相关的：下采样和上采样。在下采样中，我们创建一个与少数类大小相等的多数类的随机子集。在向上抽样时，我们反复从少数群体中抽取样本，并进行替换，使其与多数群体的规模相等。使用降采样和上采样之间的决定是根据具体情况而定的，一般来说，我们应该同时尝试两种方法，看看哪种方法能产生更好的结果。